In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import json
from scipy import ndimage


from readImages import *
from imageModifiers import *
from readDataFiles import *
from projEdgeBreaks import *
from gaussBreaks import *
from saveImages import *
from plottingFuncs import *
from gap_metric_breaks import *

mpl.rcParams["figure.figsize"] = (15, 15)
# %matplotlib inline

In [2]:
subjFile = "data/anti-slavery-manuscripts-subjects.csv"
subj = pd.read_csv(subjFile)

# get metadata in dictionary format
subj["meta_json"] = [json.loads(q) for q in subj["metadata"]]

# get image url
subj["url"] = [json.loads(q).get("0") for q in subj["locations"]]

# get beta images
subj_beta = subj.loc[subj["subject_set_id"] == 16228]
len(subj_beta)

60

In [3]:
# read in and do all pre-processing #################################
let_orig, grey_orig = readImg(subj_beta["url"].iloc[10])
grey, let, offset = removeEdges(grey_orig, let_orig, 10.0)

Functions for connected component stuff

In [4]:
def boundsAndRecs(labels, osli, buffer = 0):
    rec = []
    bounds = []
    sh = np.max(labels.shape)
    for sl in osli:
        sl0 = sl[0].indices(sh)
        sl1 = sl[1].indices(sh)

        xLeng = sl1[1]-sl1[0]
        yLeng = sl0[1]-sl0[0]
#         if xLeng*yLeng > 100:
        rec.append([[sl1[0], sl0[0]], xLeng, yLeng])
        bounds.append([(sl0[0]-buffer, sl0[1]+buffer),
                       (sl1[0]-buffer, sl1[1]+buffer)])
    return rec, bounds

def plotCC(labels, rec):
    loop = len(labels)
    fig, ax = plt.subplots(figsize=(27, 18), ncols=loop, nrows=1)
    for i in range(loop):
        if loop > 1:
            axf = ax.flatten()[i]
        else:
            axf = ax
        axf.imshow(labels[i], cmap='nipy_spectral')
        for j in range(len(rec[i])):
            rect = mpl.patches.Rectangle(rec[i][j][0], rec[i][j][1], rec[i][j][2],
                                         linewidth=1, edgecolor="r", facecolor="none")
            axf.add_patch(rect)
    plt.show()

# Line breaks

In [5]:
# get linebreaks ####################################################
greySm = smoothImg(grey, 7.0)
matchlim = 30
lb = projBreaks(greySm, "y")
lb = filterBreaks(lb, matchlim)
# plotBoxes(let, lb)#, saveFile="example_img/linebreaks.png")

# Get word breaks via convex hulls

In [6]:
i = 8
chunk = grey[lb[i-1]:lb[i],]

the_d, chhul = getGapDistances(chunk)

[46.00116744021029,
 43.61849114499686,
 11.999329876684113,
 7.741477784631998,
 42.3202410209937,
 31.394548770471673,
 27.591810743139302,
 16.68700585996636,
 7.2911169822055895,
 18.859493857686072,
 27.60522559760355,
 36.920852382717605,
 36.624897208774186,
 8.434357655417633,
 39.95043647844952,
 6.65604123641312,
 22.000168466724624,
 4.947543184356283,
 8.660805793980163,
 10.582184452039343,
 33.274620589186775,
 14.691977672989735,
 20.15217849105847]